In [ ]:
%%capture
pip install cobra

In [ ]:
import cobra 
PathToModel = ""
Conditions = ""
Medium = ""

In [ ]:
import csv
model = cobra.io.read_sbml_model(PathToModel)
exchange_rxns = model.exchanges
no_exchange = []
mediaForModel = {}
with open(Conditions, 'r') as inp:
  for line in inp:
    id = line.strip('\n')
    #print(id)
    if id in exchange_rxns:
      #rxns = model.exchanges[id]
      r = model.medium[id]
      mediaForModel[id] = r
      #print(r)
print (mediaForModel)

# Processing media
Keep track of media components in the model

In [ ]:
Original_medium = model.medium
Original_medium = mediaForModel
model.medium = mediaForModel
model.medium

#Change objectives
Optimize the model with FBA setting glucose uptake to -18

In [ ]:
#Input a realistic flux for glucose as substrate and see what the output is.
#model.medium['EX_glc__D_e'] = 18.0
#model.medium
model.reactions.EX_glc__D_e.lower_bound = -18
model.optimize()

#Simulation with Substrates
Open the original path for possible substrates reactions for simulations of growth rate

In [ ]:
#Parse all the original substrates found in the original model as presented by carveme from a file SUbstrates.txt already prepared
SubstratesPath = ''
Substrates = []
Final_Substrates = {}
with open(SubstratesPath, 'r') as inp:
  for line in inp:
    id = line.strip('\n')
    Substrates.append(id)
    Final_Substrates[id] = 1000
print(Final_Substrates)


In [ ]:
#Remove glucose from the model medium by assigning a lower bound of 0.0
model.medium['EX_12ppd__S_e'] = 1000
model.reactions.EX_12ppd__S_e.lower_bound = -18.0
model.medium

In [ ]:
#read previously saved model with minimal media 
model1 = cobra.io.read_sbml_model("")

In [ ]:
model1.medium

#Carbon source simulation


In [ ]:

def CarbonSource (model, Substratesfile):
  """Simulates the effects of different carbon sources on the objective of a flux balance solution. model is an xml file, substratefile is a txt file with a list of possible substrates ."""
  Simulation = {}
  model1.reactions.get_by_id('EX_glc__D_e').lower_bound = 0.0
  for i in Substratesfile:
    model.medium[i] = 1000
    model.reactions.get_by_id(i).lower_bound = -10.0
    solution = model1.optimize()
    flux = solution.objective_value
    Simulation[i] = flux
    del model.medium[i]
  print(model.medium)
  return Simulation

In [ ]:
#delete an entry in a dictionary
#del mediaForModel['EX_glc__D_e']

In [ ]:
#Cant get each carbon source to optimize on its own. 
CarbonSource(model1, Final_Substrates )

In [ ]:
import pandas
from pandas import DataFrame

##Single sources algorithm

In [ ]:
#I am creating a function that will take individual substrate elements and return the flux value.
def CarbonSource_with_SingleSubstrates (model, substrate_compound):
  """Simulates the effects of carbon sources on the objective of a flux balance solution. Model is an xml file, substratefile is a txt file with a list of possible substrates."""
  Simulation = {}
  i = substrate_compound
  model.medium = mediaForModel
  model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0.0
  model.medium[i] = 1000
  model.reactions.get_by_id(i).lower_bound = -18.0
  solution = model1.optimize()
  flux = solution.objective_value
  Simulation[i] = flux
  del model.medium[i]
  #print(model.medium)
  print(DataFrame.from_dict(Simulation, orient = 'index', columns = ["Carbon_source,  Flux_value"]))
  print(DataFrame.from_dict(model.medium, orient = 'index', columns = ["Carbon_source,  Upper_bound"]))
  return 

In [ ]:
CarbonSource_with_SingleSubstrates(model1, 'EX_12ppd__S_e' )

In [ ]:
CarbonSource_with_SingleSubstrates(model1, 'EX_15dap_e' )

In [ ]:
CarbonSource_with_SingleSubstrates(model1, 'EX_23dappa_e')

In [ ]:
CarbonSource_with_SingleSubstrates(model1, 'EX_2ddglcn_e')

##Multiple Source algorithm

In [ ]:
#Lets do a function that runs all the substrate components without having to put them individually
#I am creating a function that will take individual substrate elements and return the flux value.
def CarbonSource_with_SubstrateFile(model, SubstrateFile):
  """Simulates the effects of carbon sources on the objective of a flux balance solution. model is an xml file, substratefile is a txt file with a list of possible substrates."""
  Simulation = {}
  #i = substrate_compound
  for i in SubstrateFile:
    model.medium = mediaForModel
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0.0
    model.medium[i] = 1000
    model.reactions.get_by_id(i).lower_bound = -18.0
    solution = model1.optimize()
    flux = solution.objective_value
    Simulation[i] = flux
    del model.medium[i]
    Medium_log = DataFrame.from_dict(model.medium, orient = 'index', columns = ["Carbon_source,  Upper_bound"])#just to control to make sure only one carbon compound was used at a time. This is the medium component at the time the last substrate was run
  Simulation_results = DataFrame.from_dict(Simulation, orient = 'index', columns = ["Carbon_source,  Flux_value"])
  return Simulation_results

In [ ]:
Results = CarbonSource_with_SubstrateFile(model1, Substrates)

In [ ]:
import matplotlib.pyplot as plt

##Export multiple source result to csv file for plotting in qtiplot or excel


In [ ]:
DataFrame.to_csv(Results,'' )


In [ ]:
CarbonSource_with_SingleSubstrates(model1, 'EX_malthp_e')

##Plotting attempt with matplotlib


In [ ]:
CsvfilePath = ''

In [ ]:
#open the csv file save above to parse the contents to an x and y variable ready for plotting
with open(CsvfilePath, 'r') as inp:
  y = []
  x = []
  for line in inp:
    if line[0] == "E":
      line1 = line.strip('\n')
      id = line1.split(',')
      y.append(float(id[1]))
      x.append(id[0])
  print(x, y )

In [ ]:
#Plot x,y variable in matplot lib and get the graph
plt.plot(y, x)#Plot not too good

In [ ]:
model1.optimize()

#Redox, energy production and Fe

In [ ]:
#model1 = cobra.io.read_sbml_model('')
model1 = cobra.io.read_sbml_model('')
model1.objective = "Growth"
#to get a sense of the main energy production and consumption reactions
model1.metabolites.atp_c.summary()
#to examine the overall redox balance of the model
model1.metabolites.nadh_c.summary()
#to examine all iron consuming and producing reactions in the model
model1.metabolites.fe2_c.summary()

#Production envelopes

Production envelopes (aka phenotype phase planes) will show distinct phases of optimal growth with different use of two different substrates.

In [ ]:
import cobra.test
from cobra.flux_analysis import production_envelope

In [ ]:
#model1.medium
model1.reactions.EX_o2_e.lower_bound = -18.0
model1.reactions.EX_fe2_e.lower_bound = -18.0

In [ ]:
prod_env = production_envelope(
    model1, ["EX_o2_e"], objective="EX_sucr_e", carbon_sources="EX_glc__D_e")

In [ ]:
#oxygen
prod_env.plot(
    kind='line', x='EX_o2_e', y='carbon_yield_maximum');